In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_1_Radiotherapy-Treatment/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-17 15:41:44.374693


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the following parameters, decision variables, objective function, and constraints:

Parameters:
- n: Number of patients (n = 17)
- C: Total proton therapy capacity (C = 100)
- BED_ij: Biological Equivalent Dose score for patient i when using j proton fractions

Decision Variables:
- x_ij: Number of proton fractions assigned to patient i (integer variable, i ∈ {1, ..., n}, j ∈ {0, ..., 15})

Objective Function:
Maximize the total BED scores for all patients:

max Σ_i Σ_j BED_ij * x_ij

Constraints:

1. Each patient must receive exactly 15 fractions of treatment:
Σ_j x_ij = 15, for i ∈ {1, ..., n}

2. The total number of proton fractions used should not exceed the capacity limit:
Σ_i Σ_j j * x_ij ≤ C

3. The number of proton fractions assigned to each patient should be less than or equal to the total number of fractions (15):
x_ij ≤ 15, for i ∈ {1, ..., n}, j ∈ {0, ..., 15}

4. The decision variables x_ij should be non-negative integers:
x_ij ∈ {0, 1, ..., 15}, for i ∈ {1, ..., n}, j ∈ {0, ..., 15}

This optimization model can be solved using integer linear programming techniques. Once the model is solved, the optimal values of decision variables x_ij will provide the number of proton fractions to be assigned to each patient to maximize the total BED scores while staying within the proton therapy capacity limit.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the following parameters, decision variables, objective function, and constraints:

Parameters:
- n: Number of patients (n = 17)
- C: Total proton therapy capacity (C = 100)
- BED_ij: Biological Equivalent Dose score for patient i when using j proton fractions

Decision Variables:
- x_ij: Number of proton fractions assigned to patient i (integer variable, i ∈ {1, ..., n}, j ∈ {0, ..., 15})

Objective Function:
Maximize the total BED scores for all patients:

max Σ_i Σ_j BED_ij * x_ij

Constraints:

1. Each patient must receive exactly 15 fractions of treatment:
Σ_j x_ij = 15, for i ∈ {1, ..., n}

2. The total number of proton fractions used should not exceed the capacity limit:
Σ_i Σ_j j * x_ij ≤ C

3. The number of proton fractions assigned to each patient should be less than or equal to the total number of fractions (15):
x_ij ≤ 15, for i ∈ {1, ..., n}, j ∈ {0, ..., 15}

4. The decision variables x_ij sho

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

To solve this optimization problem using Pyomo, we will first need to install the necessary packages. You can do this by running:

```bash
pip install pyomo
pip install pyomo.solvers
```

Now, let's write the Pyomo code for this problem. In this code, we will use sample data for `BED_ij`.

```python
import pyomo.environ as pyo

# Sample data
n = 17
C = 100
BED = {(i, j): (i + 1) * (j + 1) for i in range(n) for j in range(16)}

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers, bounds=(0, 15))

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C

model.constraint2 = pyo.Constraint(rule=constraint2_rule)

def constraint3_rule(model, i, j):
    return model.x[i, j] <= 15

model.constraint3 = pyo.Constraint(model.I, model.J, rule=constraint3_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Total BED score:", pyo.value(model.objective))
for i in model.I:
    print(f"Patient {i}: {[model.x[i, j].value for j in model.J]}")
```

This code creates a Pyomo model for the given optimization problem and solves it using the GLPK solver. The results display the total BED score and the number of proton fractions assigned to each patient.

In [10]:
print(response_2_text)

To solve this optimization problem using Pyomo, we will first need to install the necessary packages. You can do this by running:

```bash
pip install pyomo
pip install pyomo.solvers
```

Now, let's write the Pyomo code for this problem. In this code, we will use sample data for `BED_ij`.

```python
import pyomo.environ as pyo

# Sample data
n = 17
C = 100
BED = {(i, j): (i + 1) * (j + 1) for i in range(n) for j in range(16)}

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers, bounds=(0, 15))

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1_rule)



### Code Executability

In [11]:
import pyomo.environ as pyo

# Sample data
n = 17
C = 100
BED = {(i, j): (i + 1) * (j + 1) for i in range(n) for j in range(16)}

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers, bounds=(0, 15))

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C

model.constraint2 = pyo.Constraint(rule=constraint2_rule)

def constraint3_rule(model, i, j):
    return model.x[i, j] <= 15

model.constraint3 = pyo.Constraint(model.I, model.J, rule=constraint3_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Total BED score:", pyo.value(model.objective))
for i in model.I:
    print(f"Patient {i}: {[model.x[i, j].value for j in model.J]}")

2024-05-17 15:54:13,354 ERROR pyomo.core: Rule failed when generating expression for Objective objective with index None:
KeyError: (17, 0)
2024-05-17 15:54:13,355 ERROR pyomo.core: Constructing component 'objective' from data=None failed:
KeyError: (17, 0)


KeyError: (17, 0)

### Solution Correctness

In [21]:
import pyomo.environ as pyo

# Sample data
n = 17
C = 100
BED = {}
with open(FILE_PATH_PROBLEM + 'ProblemData.csv', 'r') as f:
    for i, line in enumerate(f):
        for j, val in enumerate(line.strip().split(',')):
            BED[(i + 1, j)] = float(val)

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers, bounds=(0, 15))

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C

model.constraint2 = pyo.Constraint(rule=constraint2_rule)

def constraint3_rule(model, i, j):
    return model.x[i, j] <= 15

model.constraint3 = pyo.Constraint(model.I, model.J, rule=constraint3_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Total BED score:", pyo.value(model.objective))
for i in model.I:
    print(f"Patient {i}: {[model.x[i, j].value for j in model.J]}")

Total BED score: 85.10000000000001
Patient 1: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 2: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 3: [0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 4: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 5: [4.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 6: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 7: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 8: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 9: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 10: [15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Patient 11: [15.0, 0.0, 0.0, 0.0, 